In [20]:
from ortools.linear_solver import pywraplp

In [21]:
solver = pywraplp.Solver.CreateSolver ("SCIP")

In [22]:
# INDICES 

qtd_itens = 10                # i = 1, ..., N 10 itens
qtd_periodos = 4              # t = 1,..., T 4 semanas
qtd_subperiodos = 10         # s = 1, ..., S 10 subperiodos em cada periodo t 

##########################################################################################################################

# PARAMETROS

# MATRIZ DE DEMANDAS DOS PRODUTOS POR PERIODOS
mat_demandas = [
    [540, 489, 235, 341, 332, 441, 272, 388, 364, 109],
    [445, 382, 213, 257, 306, 332, 453, 459, 611, 140],
    [526, 423, 435, 338, 282, 339, 313, 150, 669, 204],
    [470, 395, 309, 357, 197, 319, 450, 529, 409, 167]]                                     #dit

vet_custo_estoque = [0.38, 0.92, 0.31, 0.26, 0.07, 0.45, 0.17, 0.74, 0.67, 0.99]             #hi
vet_tempo_producao = [20, 20, 20, 60, 60, 30, 30, 30, 30, 30]                                #ui
qtde_minima = 25                                                                             #L  
vet_fator_seg= [0.2, 0.2, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15]                   #fseg
vet_estoque_inicial = [144, 123, 31, 50, 64, 88, 83, 64, 97, 46]                             #It0
capacidade_disp = 144000 #total de segundos na semana                                        #Rt
m_grande = 7200 #maximo de producao na semana do item mais rapido                            #M

# MATRIZ TEMPO DOS SETUPS
matriz_temposetup = [
    [0, 600, 720, 2700, 2880, 900, 1080, 1080, 900, 900],
    [600, 0, 720, 2700, 2880, 900, 1080, 1080, 900, 900],
    [660, 720, 0, 2760, 2400, 540, 1200, 1080, 960, 1140],
    [1560, 1500, 2400, 0, 780, 2100, 2100, 1380, 1380, 1500],
    [1920, 1860, 2400, 780, 0, 1920, 1860, 1860, 1920, 1800],
    [1140, 1500, 1200, 2520, 2700, 0, 360, 720, 420, 900],
    [1140, 1200, 1200, 2640, 2520, 420, 0, 720, 840, 900],
    [1140, 1080, 1320, 2640, 2460, 600, 900, 0, 480, 900],
    [1200, 1080, 1320, 2400, 2460, 420, 900, 360, 0, 900],
    [1260, 1200, 1380, 2580, 2580, 840, 960, 720, 780, 0]]

# MATRIZ CUSTO DOS SETUP
matriz_custosetup = [
    [0, 17, 21, 78, 84, 26, 31, 31, 26, 26],
    [17, 0, 21, 78, 84, 26, 31, 31, 26, 26],
    [19, 21, 0, 80, 70, 16, 35, 31, 28, 33],
    [45, 44, 70, 0, 23, 61, 61, 40, 40, 44],
    [56, 54, 70, 23, 0, 56, 54, 54, 56, 52],
    [33, 44, 35, 73, 78, 0, 10, 21, 12, 26],
    [33, 35, 35, 77, 73, 12, 0, 21, 24, 26],
    [33, 31, 38, 77, 71, 17, 26, 0, 14, 26],
    [35, 31, 38, 70, 71, 12, 26, 10, 0, 26],
    [37, 35, 40, 75, 75, 24, 28, 21, 23, 0]]


##########################################################################################################################

#VARIAVEIS DE DECISÃO

# Variáveis de decisão de quantidade a ser produzida
producao = list()
for t in range(qtd_periodos):
    linha=list()
    for s in range(qtd_subperiodos):
        variavel=list()
        linha.append(variavel)
        for i in range(qtd_itens):
            variavel.append(solver.IntVar(0, m_grande, f"X_{t}_{s}_{i}"))
    producao.append(linha)

#Variáveis de decisão de estoque
estoque= list()
for t in range(qtd_periodos):
    variavel=list()
    for i in range(qtd_itens):
        variavel.append(solver.IntVar(0,m_grande, f"I_{t}_{i}"))
    estoque.append(variavel)


#Variáveis de decisão de produção
z = list()
for t in range(qtd_periodos):
    linha=list()
    for s in range(qtd_subperiodos):
        variavel=list()
        linha.append(variavel)
        for i in range(qtd_itens):
            variavel.append(solver.IntVar(0, 1, f"Z_{t}_{s}_{i}"))
    z.append(linha)
    
#Variáveis de decisão de setup
y = list()
for t in range(qtd_periodos):
    linha=list()
    for s in range(qtd_subperiodos):
        linha2=list()
        linha.append(linha2)
        for i in range(qtd_itens):
            variavel=list()
            linha2.append(variavel)
            for j in range(qtd_itens):
                variavel.append(solver.IntVar(0, 1, f"Y_{t}_{s}_{i}_{j} "))
    y.append(linha)

In [23]:
# ROTULOS DOS ITENS E PERIODOS

itens = ["X{}".format(i + 1) for i in range(qtd_itens)]
periodos = [t + 1 for t in range(qtd_periodos)]
subperiodos = [s + 1 for s in range(qtd_subperiodos)]

In [24]:
# FUNÇÃO OBJETIVO (0)
objetivo = (sum(vet_custo_estoque[i] * estoque[t][i] for t in range(qtd_periodos) for i in range(qtd_itens))
+ sum(matriz_custosetup[i][j] * y[t][s][i][j] 
                   for t in range(qtd_periodos)
                   for s in range(qtd_subperiodos)
                   for i in range(qtd_itens)
                   for j in range(qtd_itens)))                                          # oook!!!



solver.Minimize(objetivo)

In [25]:
# BALANÇO ESTOQUE (1)

# t = 0
for i in range(qtd_itens):
        solver.Add(sum(producao[0][s][i] for s in range(qtd_subperiodos))
                   + vet_estoque_inicial[i] - mat_demandas [0][i] == estoque [0][i])   #oook!!!
    
# t >0
for t in range(1, qtd_periodos):
    for i in range(qtd_itens):
        solver.Add(sum(producao[t][s][i] for s in range(qtd_subperiodos))
                   + estoque[t-1][i] - mat_demandas [t][i] == estoque [t][i])          #oook!!! 

In [26]:
# RESTRIÇÃO DE CAPACIDADE (2)
for t in range(qtd_periodos):
    solver.Add(sum((producao[t][s][i] * vet_tempo_producao[i] for s in range(qtd_subperiodos) for i in range(qtd_itens)))
                   + sum((y[t][s][i][j] * matriz_temposetup[i][j]
                      for s in range(qtd_subperiodos)
                      for i in range(qtd_itens)
                      for j in range(qtd_itens))) <= capacidade_disp)                #oook!!!
    

In [27]:
# RESTRIÇÃO DE PRODUÇÃO MÁXIMA DE X (3)
for t in range(qtd_periodos):
    for i in range(qtd_itens):
        for s in range(qtd_subperiodos):
            solver.Add(producao[t][s][i] <= m_grande * z[t][s][i]) #ou y[t][s][j][i]    #oook!!!
    

In [28]:
# RESTRIÇÃO DE PRODUÇÃO MÍNIMA DE X (4)
for t in range(qtd_periodos):
    for i in range(qtd_itens):
        for s in range(qtd_subperiodos):
            solver.Add(producao[t][s][i] >= qtde_minima * z[t][s][i])       #oook!!!

In [29]:
# RESTRIÇÃO DE ESTOQUE DE SEGURANÇA (5)
for t in range(qtd_periodos):
    for i in range(qtd_itens):
        solver.Add(estoque[t][i] >= vet_fator_seg[i] * mat_demandas[t][i] )  #oook!!!

In [30]:
# RESTRIÇÃO QUE GARANTE A PRODUÇÃO DE UM UNICO ITEM NO SUBPERIODO (6)
for t in range(qtd_periodos):
    for s in range(qtd_subperiodos):
            solver.Add(sum(z[t][s][i] for i in range(qtd_itens)) == 1)  #oook!!!
        

#RESTRIÇÃO QUE GARANTE A ENTRADA DE UM UNICO ITEM NO SUBPERIODO (7)
for t in range(qtd_periodos):
    for s in range(qtd_subperiodos):
        for j in range(qtd_itens):
            solver.Add(sum(y[t][s][i][j] for i in range(qtd_itens)) <= 1) #ook!!! 

# RESTRIÇÃO QUE GARANTE A SAÍDA DE UM UNICO ITEM NO SUBPERIODO (8)
for t in range(qtd_periodos):
    for s in range(qtd_subperiodos):
        for i in range(qtd_itens):
            solver.Add(sum(y[t][s][i][j] for j in range(qtd_itens)) <= 1)   #ook!!!     

# RESTRIÇÃO QUE GARANTE QUE SÓ ESCOLHAM SETUPS POSSÍVEIS P/ SISTEMA (9)

#para t>0 e s=0
 #y_ijs> = z_(i,s-1) + z_js - 1
for t in range(1, qtd_periodos):
    for s in range(0,0):
        for i in range(qtd_itens):
            for j in range(qtd_itens):
                solver.Add(y[t][s][j][i] >= z[t-1][9][j] + z[t][0][i] - 1)    #???? #PENSAR!!! 

# RESTRIÇÃO QUE GARANTE QUE SÓ ESCOLHAM SETUPS POSSÍVEIS P/ SISTEMA (9)

#para s>0
 #y_ijs> = z_(i,s-1) + z_js - 1
for t in range(qtd_periodos):
    for s in range(1, qtd_subperiodos):
        for i in range(qtd_itens):
            for j in range(qtd_itens):
                solver.Add(y[t][s][j][i] >= z[t][s-1][j] + z[t][s][i] - 1)    #???? #PENSAR!!!                  



In [ ]:
# Roda o solver e verifica se chegou a uma solução ótima
solver.set_time_limit(86400000)
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print("Solução ótima encontrada!")
    print(objetivo.solution_value())
else:
    print("Não foi encontrada solução ótima")
    print("Solução obtida:", objetivo.solution_value())
    # Melhor solução obtida
    valor_final = objetivo.solution_value()
    # Melhor solução teórica
    limitante_inferior = solver.Objective().BestBound()
    print("Limitante inferior:", limitante_inferior)
    # Gap é a diferença relativa entre os dois
    gap = 100 * (valor_final - limitante_inferior) / limitante_inferior
    print(f"Gap: {gap:.02f} %")

In [ ]:
import time
import timeit
inicio = time.time()

# Resolve o problema
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print("Solução ótima encontrada!")
else:
    print("Não foi encontrada a solução ótima")
    
fim = time.time()
print(fim - inicio)

In [ ]:
# Valor objetivo final
print(objetivo.solution_value())

In [ ]:
# Imprime a produção no subperiodos s
for t in range(qtd_periodos):
    print(f" Período: {t}")
    for s in range(qtd_subperiodos):
        for i in range(qtd_itens):
                production = round(producao[t][s][i] . solution_value())
                if production > 0:
                    print(f" {production} do  {itens[i]} no subperiodo {s}")
                    #print(production,itens[i])

In [ ]:
#Imprime estoque no final do período
for t in range(qtd_periodos):
    print(f" Fim do Período: {t}")
    for i in range(qtd_itens):
        estoqueee = round(estoque[t][i]. solution_value())
        print(f" O estoque do item {i} é {estoqueee}")

In [ ]:
# Imprime a capacidade utilizada no período t

for t in range(qtd_periodos):
    print(f" Período: {t}")
    exp_capacidade = (sum(producao[t][s][i] * vet_tempo_producao[i] 
                     for s in range(qtd_subperiodos)
                     for i in range(qtd_itens)) + 
                     sum(y[t][s][i][j] * matriz_temposetup [i][j] 
                     for s in range(qtd_subperiodos) 
                     for i in range(qtd_itens) 
                     for j in range(qtd_itens)))
    print(f"Capacidade Total Utilizada: {exp_capacidade.solution_value()}")

In [ ]:
# Imprime os valores assumidos pela variavél binária de produção Z 
for t in range(qtd_periodos):
    print(f" Período: {t}")    
    for s in range(qtd_subperiodos):
        for i in range(qtd_itens):
            produzir = round(z[t][s][i]. solution_value())
            if produzir >0:
                print(f" sub {s} {itens[i]} , {produzir}")

In [ ]:
# Imprime os valores assumidos pela variável binária de setup Y
for t in range(qtd_periodos):
    print(f" Período: {t}")
    for s in range(qtd_subperiodos):
        for i in range(qtd_itens):
            for j in range(qtd_itens):
                setup = round(y[t][s][i][j]. solution_value())
                if setup ==1:
                    print(f" Do item {itens[i]} para {itens[j]} no sub {s}")